In [1]:
import tensorflow as tf
import numpy as np
import os
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
import data_prj as dp

batch_size = 64
epochs = 6
learning_rate = .001

panneaux_id, panneaux_image = dp.get_panneaux(dp.rep_panneaux, dp.panneau_shape)
nombre_classes = len(panneaux_id)

model = dp.create_model2(dp.panneau_shape, nombre_classes)

"""
model=Sequential()

model.add(layers.Input(shape = (panneau_shape[0], panneau_shape[1], 3), dtype='float32'))

model.add(layers.Conv2D(128, 3, strides=1))
model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(128, 3, strides=1))
model.add(layers.Dropout(0.2))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPool2D(pool_size=2, strides=2))

model.add(layers.Conv2D(256, 3, strides=1))
model.add(layers.Dropout(0.3))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.Conv2D(256, 3, strides=1))
model.add(layers.Dropout(0.4))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))

model.add(layers.MaxPool2D(pool_size=2, strides=2))

model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.BatchNormalization())

model.add(layers.Dense(nombre_classes, activation='softmax'))
"""


tab_images = np.array([]).reshape(0, dp.panneau_shape[0], dp.panneau_shape[1], 3)

nbre_images = 0
for no_image, image in enumerate(panneaux_image) :
    gen_images = dp.generate_images(image, 15000)
    tab_images = np.concatenate([tab_images, gen_images])
    nbre_images += len(gen_images)
    label_hot_encoded = np.zeros(nombre_classes)
    label_hot_encoded[no_image] = 1
    if no_image :
        tab_labels = np.concatenate([tab_labels, np.full((len(gen_images), nombre_classes), label_hot_encoded)])
    else :
        tab_labels = np.full((len(gen_images), nombre_classes), label_hot_encoded)
print("Nombre de paneaux : ", nbre_images)

tab_images = np.array(tab_images, dtype=np.float32) / 255

tab_images, tab_labels = shuffle(tab_images, tab_labels)
train_images, test_images, train_labels, test_labels = train_test_split(tab_images, tab_labels, test_size=0.15)

train_data = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
test_data = tf.data.Dataset.from_tensor_slices((test_images, test_labels))

train_batch = train_data.shuffle(len(train_data)).batch(batch_size)
test_batch = test_data.shuffle(len(test_data)).batch(batch_size)

model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = tf.keras.losses.CategoricalCrossentropy(),
              metrics = [tf.keras.metrics.CategoricalAccuracy()])

history = model.fit(train_batch, epochs = epochs, validation_data = test_batch)

model.save("model2.h5")

Nombre de paneaux :  90000
Epoch 1/6
1196/1196 [==============================] - 103s 80ms/step - loss: 0.5392 - categorical_accuracy: 0.8087 - val_loss: 0.2313 - val_categorical_accuracy: 0.9316
Epoch 2/6
1196/1196 [==============================] - 95s 79ms/step - loss: 0.2747 - categorical_accuracy: 0.9073 - val_loss: 0.2307 - val_categorical_accuracy: 0.9308
Epoch 3/6
1196/1196 [==============================] - 95s 79ms/step - loss: 0.1486 - categorical_accuracy: 0.9501 - val_loss: 0.1691 - val_categorical_accuracy: 0.9430
Epoch 4/6
1196/1196 [==============================] - 95s 79ms/step - loss: 0.1336 - categorical_accuracy: 0.9549 - val_loss: 0.1308 - val_categorical_accuracy: 0.9587
Epoch 5/6
1196/1196 [==============================] - 95s 79ms/step - loss: 0.0940 - categorical_accuracy: 0.9686 - val_loss: 0.3135 - val_categorical_accuracy: 0.9003
Epoch 6/6
1196/1196 [==============================] - 94s 79ms/step - loss: 0.0899 - categorical_accuracy: 0.9700 - val_loss: 